In [1]:
import numpy             as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.optimize      import fsolve, curve_fit
from scipy.integrate     import odeint, quad
from scipy.interpolate   import interp1d
from scipy.misc          import derivative
import scipy.constants   as const
import sympy             as sp
sp.init_printing()
import glob
import math
import alpaca_trade_api as tradeapi
from alpaca_trade_api import StreamConn
import threading
import time
import datetime
import logging
import argparse
import time
import pandas as pd
ALPACA_BASE_URL ='https://paper-api.alpaca.markets';
KEY = 'PKRK5NERF9UZD8JKN80K'
SCRKEY = 'A6tH776v1Jt99Qas59V4PlXlkH9dpeeaRycyEyF9'

from lxml import html
import requests
import json
import argparse
from collections import OrderedDict
from alpha_vantage.timeseries import TimeSeries
import talib as tlb
import scipy.stats as stat

### FINDING THE BEST STOCKS 

In [2]:
def simulator(symbol,period=20,RSI_period=14,RSI_l_p=100,nup=1.8,ndwn=1.2,num=390,RSI_trigger_buy=20, delay=1,printing=True,plotting=False):
    #BUFFER ZONE
    buffer=max(period,RSI_period+RSI_l_p)
    num+=buffer
    if num>1000:
        num=1000
    
    #OHLC
    print(symbol)
    api    = tradeapi.REST(KEY, SCRKEY, ALPACA_BASE_URL, api_version='v2')
    sb     = api.get_barset(symbol, 'minute', limit = num).df
    prices = {}
    A      = ['close','open','high','low','volume']
    for a in A:
        prices[a]=sb[symbol][a]
    for i in range(len(prices['close'])):
        if prices['close'][i]==0:
            prices['close'][i]=prices['close'][i-1]
    
    
    #BOLLINGER BANDS    
    up,mid,lwr = tlb.BBANDS(prices['close'],period,nbdevup=nup,nbdevdn=ndwn,matype=1)
    up,mid,lwr = up[buffer:],mid[buffer:],lwr[buffer:]
    
    #RSI
    RSI = tlb.RSI(prices['close'], timeperiod=RSI_period)
    #RSI = RSI[RSI_period:]
    levels = [1,5,10,25,50,75,90,95,99]
    B = stat.scoreatpercentile(RSI, levels)
    RSI_level = []
    for i,current in enumerate(RSI,start=0):
        RSI_level.append(stat.percentileofscore(RSI[i:i+RSI_l_p],current)) #!!! period of RSI? 
    RSI=RSI[buffer:]
        
    ##SIMULATOR
    buy  = np.zeros(num-period)
    sell = np.zeros(num-period)
    quantity = 0
   
    for i in range(num-buffer):  
        if up[i]<=prices['close'][i+buffer] and RSI_level[i]<80:
            buy[i]=prices['close'][i+period+delay]*2
            quantity+=2
        
        if RSI_level[i]<RSI_trigger_buy and lwr[i]<=prices['close'][i+buffer]:
            buy[i]=buy[i]+prices['close'][i+buffer+delay]*2
            quantity+=2
            
        if prices['close'][i+buffer]<lwr[i] and quantity>0:
            sell[i]=prices['close'][i+buffer+delay]*quantity
            quantity=0
            
       # if RSI_level[i]>90 and quantity>0:
       #     sell[i]=sell[i]+prices['close'][i+period]*quantity
       #     quantity=0
 
    #cash
    cash = []
    cash.clear()
    for i in range(len(buy)):
        cash.append(sell[i]-buy[i])

    cash_profit = sum(sell)-sum(buy)
    print('cash profit =', cash_profit)
    
    profit = cash_profit+quantity*prices['close'][-1]
    print('The profit=', profit)
    
    #cumulative cash flow
    cum_cash = []
    cum_cash.clear()
    for i in range(len(buy)):
        cum_cash.append(sum(cash[0:i]))
    
    #RETURN ON MONEY
    if min(cum_cash)==0:
        Rom=0
    else:
        Rom = -profit/min(cum_cash)
    print('ROM = ',Rom*100, '%') #
    
        
        #PLOTTING
    if printing ==True:
        t = np.linspace(0,len(up),len(up))
        w,h = 20,10
        if plotting:
            
                 #plotting the profit
            plt.figure(figsize=(w,h))
            plt.title('cum cash flow', fontsize=20)
            plt.plot(cum_cash)
            plt.xticks(np.linspace(0,len(up),20),fontsize=20,rotation=45)
            plt.yticks(fontsize=20)
            plt.xlim(period,t[-1])
            plt.legend(fontsize=20)
            plt.grid()
            
                #plotting the profit
            plt.figure(figsize=(w,h))
            plt.title('cash flow', fontsize=20)
            plt.plot(cash)
            plt.xticks(np.linspace(0,len(up),20),fontsize=20,rotation=45)
            plt.yticks(fontsize=20)
            plt.xlim(period,t[-1])
            plt.legend(fontsize=20)
            plt.grid()
            
            plt.figure(figsize=(w,h))
            plt.plot(t,prices['close'][buffer:],label='close')
            plt.plot(t,up,'.',color='green',label='up')
            plt.plot(t,mid,'.',color='orange',label='mid')
            plt.plot(t,lwr,'.',color='red',label='low')
            plt.legend(fontsize=20)
            plt.xticks(np.linspace(0,len(up),20),fontsize=20,rotation=45)
            plt.yticks(fontsize=20)
            plt.xlim(period,t[-1])
            plt.grid()
             
            plt.figure(figsize=(w,h))
            plt.plot(t,RSI,label='RSI')
            plt.yticks(fontsize=20)
            plt.xlim(period,t[-1])
            plt.legend(fontsize=20)
            plt.grid()
            
            plt.figure(figsize=(w,h))
            plt.plot(t,RSI_level[buffer:],label='RSI')
            for i,b in enumerate(B,0):
                plt.plot(t,np.ones(len(t))*b,label=f'{levels[i]} % Percentil' )
            plt.xticks(np.linspace(0,len(up),20),fontsize=20,rotation=45)
            plt.yticks(fontsize=20)
            plt.xlim(period,t[-1])
            #plt.legend(fontsize=20)
            plt.grid()
            
            plt.figure(figsize=(w,h))
            plt.plot(t,prices['volume'][buffer:],label='volume',color='green')
            plt.xticks(np.linspace(0,len(up),20),fontsize=20,rotation=45)
            plt.yticks(fontsize=20)
            plt.xlim(period,t[-1])
            plt.legend(fontsize=20)
            plt.grid();
    
    df = pd.DataFrame(np.array([buy,sell,cum_cash]).T, columns=['buy','sell','cumulative cash flow'])
    return Rom

In [ ]:
r=simulator('TSLA',printing=True)

In [7]:
'''RETRIEVER'''
api = tradeapi.REST(KEY, SCRKEY, ALPACA_BASE_URL, api_version='v2')
stocks = api.list_assets(status='active')

NYSE_assets_full = [i for i in stocks if i.exchange=='NYSE' or 'NASDAQ']

stocks = []
for i in NYSE_assets_full:
    stocks.append(i.symbol)

'''CALLER'''
num_stocks = 0
Sum = 0
ROM = {}
for s in stocks:
    try:
        api    = tradeapi.REST(KEY, SCRKEY, ALPACA_BASE_URL, api_version='v2')
        sb     = api.get_barset(s, 'day', limit = 5).df
        A      = ['close','open','high','low','volume']
        v      = sum(sb[s]['volume'])/5
    except:
        v=0

    if v>500000:
        try:
            rom=simulator(s,
              period=25,
              RSI_period=25,
              RSI_l_p=120,
              nup=2.1,
              ndwn=1.6,
              num=970,
              RSI_trigger_buy=10,
              delay=1,
              printing=True,
              plotting=False)
            ROM[s] = rom 
            Sum +=rom
            num_stocks+=1
        except:
            ROM[s] = 0
            Sum+=0
            num_stocks+=0
            print('failed')
        print()
        #print(Sum)
print('ave profit =', Sum/len(stocks)*100,'%')
symbol='TSLA'


FE
cash profit = 6.490000000000691
The profit= 6.490000000000691
ROM =  0.5207455728603047 %

ALLY
failed

ANF
failed

HUM
cash profit = 7.140000000043074
The profit= 7.140000000043074
ROM =  0.027991854919247317 %

MYOV
cash profit = -7.349999999999454
The profit= -7.349999999999454
ROM =  -0.787798237904291 %

ODFL
cash profit = -0.8100000000194996
The profit= -0.8100000000194996
ROM =  -0.004655370400922221 %

RDN
cash profit = 21.750000000004547
The profit= 21.750000000004547
ROM =  0.9626492106278488 %

SQ
failed

TOL
failed

UL
cash profit = -24.030000000000655
The profit= -24.030000000000655
ROM =  -0.9987572683178505 %

VB


<ipython-input-7-e52b30d01529>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rom=simulator(s,
<ipython-input-2-8833b2744514>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices['close'][i]=prices['close'][i-1]


cash profit = -2980.350000000006
The profit= 588.1299999999942
ROM =  2.4768572428480513 %

TMCI
failed

IMPL
failed

SWIM
failed

RAIN
failed

FAST
cash profit = 47.42000000000189
The profit= 47.42000000000189
ROM =  0.9989340770919661 %

TRKA
failed

AACQ
cash profit = -84.62999999999647
The profit= -4.389999999996476
ROM =  -0.45651653962506145 %

AAL
cash profit = -10.140000000003965
The profit= -10.140000000003965
ROM =  -0.7586470046913392 %

AAP
cash profit = 67.33999999998923
The profit= 67.33999999998923
ROM =  0.2871497297790099 %

AAPL
cash profit = 126.83000000000175
The profit= 126.83000000000175
ROM =  0.6885955268165784 %

AAU
cash profit = -10.541200000000032
The profit= -0.5098000000000322
ROM =  -2.356737365705876 %

AAXJ
cash profit = -264.5100000000166
The profit= 114.0699999999834
ROM =  1.717396639280211 %

ABBV
cash profit = 5.7299999999886495
The profit= 5.7299999999886495
ROM =  0.04590511181050122 %

ABC
failed

ABCL
cash profit = 48.0699999999988
The profit= 

In [4]:
ROM_c=ROM
for i in dict(ROM_c):
    if pd.notnull(ROM_c[i])== False:
        del ROM_c[i] 

In [5]:
data = pd.DataFrame.from_dict(ROM,orient='index',columns=['ROM'])
Data=data.multiply(100)
sorted=Data.sort_values(by='ROM')
print(len(sorted))
a=2300

sorted[a:a+100]

2353


,ROM
CURI,6.535535
GOGL,6.557675
SYPR,6.669014
HCHC,6.881612
USA,6.906470
NMTR,6.941931
HGEN,7.045980
FNGU,7.105018
KALV,7.165503
TRCH,7.323430


In [6]:
df = sorted
with open('simulation_test.csv', 'w') as file:
    df.to_csv('simulation_test.csv')

### OPTIMIZATION OF VARIABLES 

In [ ]:
stock_list = pd.read_csv('picked.csv', names=['index','symbol','index_2','volatility'])
stocks=stock_list['symbol']
stocks=stocks.drop(labels=[0],axis=0)

'''CALLER'''
num_stocks = 0
Sum = 0
ROM = {}
for s in stocks:
    try:
        api    = tradeapi.REST(KEY, SCRKEY, ALPACA_BASE_URL, api_version='v2')
        sb     = api.get_barset(s, 'day', limit = 5).df
        A      = ['close','open','high','low','volume']
        v      = sum(sb[s]['volume'])/5
    except:
        v=0

    if v>500000:
        try:
            rom=simulator(s,
              period=25,
              RSI_period=25,
              RSI_l_p=120,
              nup=2.1,
              ndwn=1.6,
              num=390, #970
              RSI_trigger_buy=10,
              delay=1,
              printing=True,
              plotting=False)
            ROM[s] = rom 
            Sum +=rom
            num_stocks+=1
        except:
            ROM[s] = 0
            Sum+=0
            num_stocks+=0
            print('failed')
        print()
        #print(Sum)
print('ave profit =', Sum/len(stocks)*100,'%')

In [ ]:
rom=simulator('VXX',
  period=25,
  RSI_period=25,
  RSI_l_p=120,
  nup=2.1,
  ndwn=1.6,
  num=390,
  RSI_trigger_buy=10,
  delay=1,
  printing=True,
  plotting=True)